<a href="https://colab.research.google.com/github/aabelatilaw/COVID-19-Prediction-Using-Artificial-Intelligence/blob/main/Country_Wise_Confirmed_COVID_19_Cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go


# Check if any GPUs are available
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Set memory growth only if GPUs are detected
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Memory growth set for GPU:", physical_devices[0])  # Confirmation message
else:
    print("No GPUs detected. Using CPU instead.")

No GPUs detected. Using CPU instead.


# Importing Data

In [4]:
!apt-get install git
!git config --global user.email "abelatilaw4@gmail.com"
!git config --global user.name "ABEL ATILAW"
!git clone https://github.com/aabelatilaw/COVID-19-Prediction-Using-AI



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'COVID-19-Prediction-Using-AI'...


In [6]:
#confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
from google.colab import files
uploaded = files.upload()

Saving time_series_covid19_confirmed_global.csv to time_series_covid19_confirmed_global.csv


# Date Pre-Processing

In [7]:
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
confirmed_df = confirmed_df.groupby ("Country/Region").sum().reset_index()
Country_confirmed =  confirmed_df['Country/Region'].values
confirmed_df = confirmed_df.drop(['Lat','Long','Country/Region'], axis=1)
confirmed_df = confirmed_df[confirmed_df.columns].T
confirmed_df.columns = Country_confirmed
Date_index = confirmed_df.index

# Convert all columns to numeric, coerce errors to NaN
confirmed_df = confirmed_df.apply(pd.to_numeric, errors='coerce')
confirmed_df.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
Province/State,0,0,0,0,0,0,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
1/22/20,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1/23/20,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,2,0,0,0,0,0
1/24/20,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,2,0,0,0,0,0
1/25/20,0,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,2,0,0,0,0,0


In [8]:
print(confirmed_df)

                Afghanistan  Albania  Algeria  Andorra  Angola  Antarctica  \
Province/State            0        0        0        0       0           0   
1/22/20                   0        0        0        0       0           0   
1/23/20                   0        0        0        0       0           0   
1/24/20                   0        0        0        0       0           0   
1/25/20                   0        0        0        0       0           0   
...                     ...      ...      ...      ...     ...         ...   
3/5/23               209390   334427   271477    47875  105277          11   
3/6/23               209406   334427   271477    47875  105277          11   
3/7/23               209436   334427   271490    47875  105277          11   
3/8/23               209451   334443   271494    47890  105288          11   
3/9/23               209451   334457   271496    47890  105288          11   

                Antigua and Barbuda  Argentina  Armenia   Austr

# Scalling the data

In [9]:
sc = RobustScaler()
confirmed_df_scaled = sc.fit_transform(confirmed_df)
confirmed_df_scaled = pd.DataFrame(confirmed_df_scaled)

# Enter the number of days you want to predict

In [11]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))

# Instead of 'closed', use 'inclusive' with 'right' for the same behavior
predict_confirm_index = pd.date_range(start=Date_index[-1], periods=DAYS_TO_PREDICT + 1, inclusive='right')

# Alternative: Calculate the 'end' date and use it
# from datetime import timedelta
# end_date = pd.to_datetime(Date_index[-1]) + timedelta(days=DAYS_TO_PREDICT)
# predict_confirm_index = pd.date_range(start=Date_index[-1], end=end_date)

predict_confirm_index = predict_confirm_index.strftime('%Y/%m/%d')

Enter Number Of Days You Want to Predict : 1


# Creating class for building model

In [12]:
class Corona_Confirmed_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')

        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))


        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        confirm_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])

        optimizer = keras.optimizers.Adadelta(lr=0.001)

        confirm_model.compile(loss=['mae'],optimizer=optimizer)

        confirm_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)


        confirm_test_seq  = trainX[-1:]
        confirm_test_seq = confirm_test_seq
        confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq), time_steps, 1)
        confirm_preds = []

        for _ in range (DAYS_TO_PREDICT):
            confirm_pred = confirm_model.predict(confirm_test_seq)
            confirm_preds.append(confirm_pred)
            confirm_new_seq = confirm_pred
            confirm_test_seq = confirm_new_seq
            confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq ), time_steps, 1)



        return confirm_preds

In [13]:
class Corona_Confirmed_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')

        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))


        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        confirm_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])

        # Change 'lr' to 'learning_rate'
        optimizer = keras.optimizers.Adadelta(learning_rate=0.001)

        confirm_model.compile(loss=['mae'],optimizer=optimizer)

        confirm_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)


        confirm_test_seq  = trainX[-1:]
        confirm_test_seq = confirm_test_seq
        confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq), time_steps, 1)
        confirm_preds = []

        for _ in range (DAYS_TO_PREDICT):
            confirm_pred = confirm_model.predict(confirm_test_seq)
            confirm_preds.append(confirm_pred)
            confirm_new_seq = confirm_pred
            confirm_test_seq = confirm_new_seq
            confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq ), time_steps, 1)



        return confirm_preds

In [ ]:
NN = Corona_Confirmed_model()
confirm_predictions = []
for i in range(len(Country_confirmed)):
    result = NN.Model(confirmed_df_scaled,i)
    confirm_predictions.append(result)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

# Calling the class and predicting the cases for every individual country

# Processing the predicted data

In [ ]:
confirm_predictions = (np.array(confirm_predictions).transpose())
confirm_predictions = confirm_predictions.reshape(((DAYS_TO_PREDICT),len(Country_confirmed)))
confirm_predictions = sc.inverse_transform(confirm_predictions)
confirm_predictions = pd.DataFrame(confirm_predictions)
confirm_predictions = confirm_predictions.abs()
confirm_predictions['Dates'] = predict_confirm_index
confirm_predictions.set_index('Dates', inplace = True)
confirm_predictions.columns = Country_confirmed
confirm_predictions.head()

NameError: name 'np' is not defined

# Visualizing The Confirmed Cases For A Country As Per The User's Choice

### List of Infected Countries

In [ ]:
Infected_Countries = pd.DataFrame(Country_confirmed)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

### Select The Country

In [ ]:
Country =input('Enter the name of the country from above list you would like to see the daily cases till now : ')

## Till Date Confirm Cases on daily basis of coronavirus

In [ ]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                    xaxis_title = "Dates",
                    yaxis_title="Number Of Cases",
                     )
fig.add_trace(go.Scatter(x=confirmed_df.index,
                         y=confirmed_df[Country],
                         mode='lines+markers',
                         line=dict(color='blue', width=2)))

## Predictions of confirmed cases of coronavirus

In [ ]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
    xaxis_title = "Dates",
    yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirm_predictions.index,
                         y=confirm_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='yellow', width=2)))